#### 1. Organizar o código da aula 04 em funções.

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import fixed


def show_types(data):
    print(data.dtypes)
    return None


def show_dimensions(data):
    print(f'\nNumber of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}', end='\n\n')
    return None


def data_collect(path):
    data = pd.read_csv(path)
    
    show_dimensions(data)
    
    show_types(data)
    
    return data


def collect_geodata(data, cols):
    geolocator = Nominatim(user_agent='geoapiExercises')
    
    data = data.head(20)
    
    data.loc[:, cols] = 'NA'
    
    for i in range(len(data)):
        if (i+1) % 10 == 0:
            print(f'Loop: {i+1}/{len(data)}')
        
        query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
        response = geolocator.reverse(query)   
        
        for c in range(len(cols)):
            if cols[c] in response.raw['address']:    
                data.loc[i, cols[c]] = response.raw['address'][cols[c]]        
            
    return data


def data_transform(data):
    data['level'] = data['price'].apply(lambda x: '0' if x < 321950 else
                                          '1' if (x >= 321950) & (x < 450000) else
                                          '2' if (x >= 450000) & (x < 645000) else '3')
    
    
    data['is_waterfront'] = data['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')
        
    #show_dimensions(data)
    #show_types(data)
    
    #cols = ['road', 'house_number', 'neighbourhood', 'city', 'county', 'state']
    #df =  data.head(20)    
    #f1 = collect_geodata(df, cols)
    #how_dimensions(df1)
    #rint(df1.head(3))
    
    return data
    
    
def data_load(data, waterfront, limit):
    houses = data[(data['price'] <= limit) &
                  (data['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'level']].copy()
    
    fig = px.scatter_mapbox(houses,
                     lat='lat',
                     lon='long',
                     color='level',
                     size='price',
                     color_continuous_scale=px.colors.cyclical.IceFire,
                     size_max=15,
                     zoom=10)

    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(height=600, margin={'r':0, 'l':0, 'b':0, 't':0 })
    fig.show()
    
    
if __name__ == '__main__':
    data_raw = data_collect('datasets/kc_house_data.csv')

    data_processing = data_transform(data_raw)
    
waterfront_bar=widgets.Dropdown(
    options=data_processing['is_waterfront'].unique().tolist(),
    value='yes',
    description='Water View',
    disable=False)

price_limit = widgets.IntSlider(
    value = 540000,
    min = data_processing['price'].min(),
    max = data_processing['price'].max(),
    step = 1,
    description='Maximum Price',
    disable=False)

widgets.interactive(data_load, data=fixed(data_processing), waterfront=waterfront_bar, limit=price_limit)


Number of rows: 21613 
Number of columns: 21

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…